In [21]:
# get real time bitcoin prices from coinbase

import requests
import json
import pandas as pd
import time
import datetime

def get_bitcoin_price():
    # get current time
    now = datetime.datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M:%S")

    # get bitcoin price from coinbase
    r = requests.get('https://api.coinbase.com/v2/prices/BTC-TWD/spot')
    data = r.json()
    price = data['data']['amount']

    # create dataframe
    df = pd.DataFrame({'time': [now], 'price': [price]})
    df['time'] = pd.to_datetime(df['time'])

    return df['price'][0]

#real_time_prices = get_bitcoin_price()
#print(int(float(real_time_prices)))

In [22]:
# a function to creat five random numbers

import random

def get_random_numbers():
    return sorted(random.sample(range(1, 40), 5))
a = get_random_numbers()
a = [str(i) for i in a]
print(a)

['4', '7', '26', '29', '35']


In [23]:
from flask import Flask
from flask import request, abort

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from linebot.models import *
from linebot.models.template import *
from linebot import LineBotApi
from linebot.exceptions import LineBotApiError

import sqlite3

from show_lottery_numbers import *
from bitcoin_addr_checker import *
from get_bitcoint_transction import *

import datetime

api_token = pd.read_csv('C:\\Users\\user\\Desktop\\side_project\\lottery_project\\api_token.txt', header=None )
Channel_access_token = api_token[0][0]
Channel_secret = api_token[0][1]

app = Flask(__name__)


linebot_api = LineBotApi(Channel_access_token)
handler = WebhookHandler(Channel_secret)

user_numbers = {}

@app.route('/callback', methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    m_text  = event.message.text

    if m_text == '顯示比特幣價格':
        real_time_prices = get_bitcoin_price()
        linebot_api.reply_message(event.reply_token, TextSendMessage(text=\
                                "目前比特幣價格為:\n新台幣 "+str(int(float(real_time_prices)))+"元"))

    elif m_text == '顯示當期樂透號碼':
        message = show_lottery_numbers()
        linebot_api.reply_message(event.reply_token, TextSendMessage(text=message))

    elif m_text == '我的彩券':
        #連線至資料庫確認有幾張彩券
        conn = sqlite3.connect('lottery_project_DB.db')
        cur = conn.cursor()
        cur.execute("SELECT number1, number2, number3, number4, number5, date\
            FROM userID_numbers \
            WHERE userID = '{}'".format(user_id))
        rows = cur.fetchall()
        list = []
        for row in rows:
            list.append(row)
        conn.close()
        
        #還沒判斷有幾張彩券付款了
        message = '您目前有{}張有效彩券'.format(len(list))

        linebot_api.reply_message(event.reply_token, TextMessage(text=message))

    
    elif(m_text == '下注'):
        UserId = event.source.user_id
        name = linebot_api.get_profile(UserId).display_name
        message = TextSendMessage(text = '您好，'+ name +'\n請選擇選號方式',\
                                quick_reply = QuickReply(items = [\
                                    QuickReplyButton(action = MessageAction(label = "手動選號", text = "手動選號")),
                                    QuickReplyButton(action = MessageAction(label = "自動選號", text = "自動選號"))
                                    ]
                                ))
        linebot_api.reply_message(event.reply_token, message)
        

    
    elif(m_text == '手動選號'):
        linebot_api.reply_message(event.reply_token, TextSendMessage(text = '此功能尚未開放，敬請期待'))
        

    elif(m_text == '自動選號'):
        lottery_numbers = get_random_numbers()
        user_numbers[user_id] = lottery_numbers
        

        choice_message = TextSendMessage(text = '您的樂透號碼為:\n{}\n是否購買這組樂透號碼?'.format(lottery_numbers),\
                                quick_reply = QuickReply(items = [\
                                    QuickReplyButton(action = MessageAction(label = "是(確認號碼)", text = "確認號碼")),
                                    QuickReplyButton(action = MessageAction(label = "否(更換號碼)", text = "自動選號"))
                                    ]
                                ))
        linebot_api.reply_message(event.reply_token, choice_message)
        

    
    elif(m_text == '確認號碼'):
        if user_id in user_numbers:
            linebot_api.reply_message(event.reply_token, TextSendMessage(text = '請輸入您的比特幣地址'))
        
        else:
            linebot_api.reply_message(event.reply_token, TextSendMessage(text = '您尚未選擇號碼QQ\n請先回到上一步選擇號碼'))
    

    elif(check_format(m_text) ):
        if user_id in user_numbers:
            if(check_bitcoin_address(m_text)):
                localtime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                
                #將user_id、號碼、地址、時間存入資料庫
                numbers = user_numbers[user_id]
                conn = sqlite3.connect('lottery_project_DB.db')
                cur = conn.cursor()
                cur.execute('CREATE TABLE IF NOT EXISTS userID_numbers (userID TEXT, number1 TEXT, number2 TEXT, number3 TEXT, number4 TEXT, number5 TEXT, address TEXT, date TEXT)')
                cur.execute("INSERT INTO userID_numbers (userID, number1, number2, number3, number4, number5,address, date) \
                    values('{}','{}','{}','{}','{}','{}','{}','{}');".format(user_id, numbers[0], numbers[1], numbers[2], numbers[3], numbers[4], m_text, localtime))
                conn.commit()
                conn.close()
                
                linebot_api.reply_message(event.reply_token,[TextSendMessage(text = '地址格式正確!\n請確認您購買的樂透號碼:\n\
                                                                 {} , {} , {}, {} , {} '.format(numbers[0], numbers[1], numbers[2], numbers[3], numbers[4])),\
                                                            TextSendMessage(text = '請匯款至以下地址即完成彩券購買，祝您中獎!:\n「比特幣收款地址」'),\
                                                            StickerSendMessage(package_id= '446', sticker_id = '2003')])
                del user_numbers[user_id]
            else:
                linebot_api.reply_message(event.reply_token, TextSendMessage(text = '地址格式有誤!\n請重新輸入'))

        else:
            linebot_api.reply_message(event.reply_token, TextSendMessage(text = '您尚未選擇號碼QQ\n請先回到上一步選擇號碼'))  

    
    else:
        linebot_api.reply_message(event.reply_token, TextSendMessage(text='指令有誤\n請輸入正確指令^o^'))


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Feb/2023 14:28:41] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [23/Feb/2023 14:28:52] "POST /callback HTTP/1.1" 200 -
